In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Questions** (Ponder over each of them later...) 
- Can we predict company responses? Given the bounded set of subjects handled by each company, the answer seems yes!!! 
- Do requests get stale? How quickly do the best companies respond, compared to the worst? 
- Can we learn high quality dense embeddings or representations of similarity for topical clustering? 
- How does tone affect the support conversation? Does saying sorry help?: Ponder how I can analyze this...
- Can we help companies identify new problems, or ones most affecting their customers? 

In [2]:
# Data visualization 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
## Make an output "visualizations" directory for storing images 
# !cd /kaggle/working/
# !mkdir visualizations

In [1]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


: 

In [3]:
# Import necessary libraries
import re 
from collections import Counter 
import warnings 
warnings.filterwarnings("ignore")

# Object serialization 
import pickle
import sklearn

# WordCloud 
from wordcloud import WordCloud, STOPWORDS

# Data Visualization 
import matplotlib as mp
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

# Pandas pre-profiling 
from ydata_profiling import ProfileReport 

# Spelling Checker 
# from spellchecker import SpellChecker 

# Import Natural Language Processing (NLP) libraries
import nltk
# nltk.download("stopwords")
# nltk.download("wordnet")
# nltk.download("punkt")
# nltk.download("averaged_perceptron_tagger")
from nltk import word_tokenize, sent_tokenize 
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet 

# Visualization 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
# from bokeh.io import output_file, show
# from bokeh.plotting import figure
# from bokeh.models import ColorBar, ColumnDataSource
# from bokeh.colors import Color


# Import Spacy for advanced natural language processing
# import spacy

# Fasttext languae detection 
# from ftlangdetect import detect

# Contractions 
import contractions as cm 

# Import langdetect for language detection
# Note: set seed=0 to enforce consistent results (to be done later)
# from langdetect import DetectorFactory, detect 
# from spacy_langdetect import LanguageDetector
from typing import List, Dict 

# Import scikit-learn utilities
from sklearn.preprocessing import FunctionTransformer, LabelEncoder 
from sklearn.pipeline import Pipeline, FeatureUnion  

# Import Spacy tokenizer
# from spacy.tokenizer import Tokenizer 

# Import tqdm for progress bars
# Register 'pandas.progress_apply' and 'pandas.Series.map_apply' with 'tqdm' 
from tqdm import tqdm
tqdm.pandas(desc="my bar!!!")

# Chi2 test 
from scipy.stats import chi2_contingency

In [4]:
df = pd.read_pickle("../data/processed/processed_v2.pkl")
df.head()

,inbound_text,author_id,created_at,outbound_text,response_tweet_id,inbound_lang,inbound_hashtags,outbound_hashtags,clean_inbound_text,clean_outbound_text,outbound_tokens_pos,inbound_tokens_pos
4,@AmazonHelp 3 different people have given 3 di...,AmazonHelp,2017-10-31 23:28:00+00:00,@115820 We'd like to take a further look into ...,619,en,[],[],different people have given different answers ...,wed like to take a further look into this with...,"[-PRON-: NOUN, d: VERB, like: VERB, to: NOUN, ...","[different: NOUN, people: NOUN, have: NOUN, gi..."
5,Way to drop the ball on customer service @1158...,AmazonHelp,2017-10-31 22:29:00+00:00,@115820 I'm sorry we've let you down! Without ...,616,en,[],[],way to drop the ball on customer service so pi...,i am sorry we have let you down without provid...,"[i: NOUN, be: NOUN, sorry: NOUN, -PRON-: NOUN,...","[way: NOUN, to: NOUN, drop: VERB, the: NOUN, b..."
6,@115823 I want my amazon payments account CLOS...,AmazonHelp,2017-10-31 22:28:34+00:00,@115822 I am unable to affect your account via...,NaN,en,[],[],i want my amazon payments account closed dm me...,i am unable to affect your account via twitter...,"[i: NOUN, be: NOUN, unable: NOUN, to: NOUN, af...","[i: NOUN, want: VERB, -PRON-: NOUN, amazon: NO..."
9,@AmazonHelp @115826 Yeah this is crazy we’re l...,AmazonHelp,2017-11-01 12:53:34+00:00,@115827 Thanks for your patience. ^KM,NaN,en,[],[],yeah this is crazy were less than a week away ...,thanks for your patience km,"[thank: NOUN, for: NOUN, -PRON-: NOUN, patienc...","[yeah: NOUN, this: NOUN, be: NOUN, crazy: NOUN..."
10,@115828 How about you guys figure out my Xbox ...,AmazonHelp,2017-10-31 22:28:00+00:00,@115826 I'm sorry for the wait. You'll receive...,627,en,[],[],how about you guys figure out my xbox one x pr...,i am sorry for the wait you will receive an em...,"[i: NOUN, be: NOUN, sorry: NOUN, for: NOUN, th...","[how: NOUN, about: NOUN, -PRON-: NOUN, guy: NO..."


In [ ]:
#constants
eng_stopwords = set(stopwords.words("english"))

#settings
warnings.filterwarnings("ignore")
lemmatizer = WordNetLemmatizer()

In [ ]:
%%time
# Reading in all the Twitter data
all_data = pd.read_csv('/kaggle/input/customer-support-on-twitter/twcs/twcs.csv')

# Finding the distribution of all authors
count = all_data.groupby("author_id")["text"].count()

# Showing only the authors that appear more than 15000 times
c = count[count>15000].plot(kind='bar',figsize=(15, 5), color='#00acee')
c.set_xlabel('')
c.set_ylabel('Frequency')
plt.title('Most of the Data is AmazonHelp')

# Saving my plot
# plt.savefig('/kaggle/working/visualizations/authors.png')

plt.show()

# There are 108 unique authors that are companies and 702669 customers
authors_count = (len(all_data[all_data.inbound == False].author_id.unique()), len(all_data[all_data.inbound == True].author_id.unique()))

# Visualizing it
plt.figure(figsize =(25,5))
x = ['Unique Companies','Unique Customers']
ax = plt.barh(x, authors_count, color = '#00acee')
plt.title('There are way more unique customers than unique companies')
plt.show()

## Data Dictionary 

## Data Dictionary:
* Tweet_id: The unique ID for this tweet
* Author_id: The unique ID for this tweet author (anonymized for non-company users)
* Inbound: Whether or not the tweet was sent (inbound) to a company
* Created_at: When the tweet was created
* Text: The text content of the tweet
* Response_tweet_id: The tweet that responded to this one, if any
* In_response_to_tweet_id: The tweet this tweet was in response to, if any

In [ ]:
# All data shape
print('All data has shape {}'.format(all_data.shape))

# Converting columns to the right datatypes
all_data['text'] = all_data['text'].astype('string')
all_data['author_id'] = all_data['author_id'].astype('string')

# This function utilizes Panda's Style to helps me encode data type and 
# magnitude information to my dataframe view
show = lambda x: x.tail(5).style.set_properties(**{'background-color': 'black',                                                   
                                    'color': 'lawngreen',                       
                                    'border-color': 'white'})\
.applymap(lambda x: f"color: {'lawngreen' if isinstance(x,str) else 'cyan'}")\
.background_gradient(cmap='Blues')

show(all_data)

## Data Wrangling 
- **Requirements**: 
    - **Inbound**: Since the first step I want to achieve is intent classification and to figure out what the customer is saying to the company, I only want data that goes from the customer to the company (inbound data).
    - **Just Amazon Customer Data**: My first modeling step is to do some sort of clustering to classify the intents for each of these tweets, doing so might be easier and more fruitful if I narrow the data to just the **amazon support data**. I can expand this later to other datasets and domains. 
    
    - {{Later, I would like my chatbot to be able to respond to make *follow up* responses to my customers.}}: Investigate this 

**Note** 
- See if I can use some **topic modeling** based **similarity matching** technique to label the intents of other 
- Any classification-based approach???

In [ ]:
amazon_data = all_data[all_data["author_id"]=="AmazonHelp"]
amazon_data.head()

- Later, I would like my bot to make *follow-up* responses to my customers  

In [ ]:
amazon_data.isnull().sum()

In [ ]:
amazon_data.dtypes

In [ ]:
amazon_data["created_at"].head()

In [ ]:
# Change the data types of "author_id" and "text" to object, and "created_at" to datetime 
amazon_data.loc[ : , ["author_id", "text"]] = amazon_data.loc[ : , ["author_id", "text"]].astype("object")
amazon_data.loc[ : , "created_at"] = pd.to_datetime(amazon_data.loc[ : , "created_at"], format="%a %b %d %H:%M:%S %z %Y")

In [ ]:
amazon_data.dtypes

In [ ]:
# Shape of Amazon Data 
amazon_data.shape

There are 169840 rows of Amazon data, which is 6 percent of the entire data. These are all data from Amazon to a customer, I want the data from the customer to Amazon for each of these Amazon customer service responses, and this can be found in the reponse_tweet_id.

Now I drop the rows with a NaN value for the in response to tweet id column, meaning that the original customer inquiry was not available for that interaction

In [ ]:
# Inspect data samples where "in_response_to_tweet_id" is null
amazon_data[amazon_data["in_response_to_tweet_id"].isnull()]

In [ ]:
amazon_data.dropna(subset = ["in_response_to_tweet_id"],inplace = True)
amazon_data["inbound"].value_counts(), amazon_data.shape

**Observations** 
- The above procedure let's go of 553 rows, maybe the origin inbound requests were deleted by the poster. Since I am interested in atleast single-level request response from the company, I deleted them since they won't serve me any good purpose 

In [ ]:
!pwd
!mkdir objects 

In [ ]:
# Converting the IDs to integers to match the tweet_id column for joining 
amazon_data["in_response_to_tweet_id"] = amazon_data["in_response_to_tweet_id"].astype("int64")

# Doing an inner join to get the respective tweets back from all_data and get the response tweets 
amazon_data = pd.merge(all_data[["tweet_id", "text"]], amazon_data, left_on="tweet_id", right_on="in_response_to_tweet_id")

# Dropping the columns that doesn't encode useful information and renaming the columns nicely for interpretability 
amazon_data.drop(["tweet_id_x", "tweet_id_y", "in_response_to_tweet_id", "inbound"], axis=1, inplace=True)

# Renaming the columns for interpretability 
amazon_data.rename(columns={"text_x": "inbound_text", "text_y": "outbound_text"}, inplace=True)

# Subsetting to inbound and outboudn text 
text = amazon_data[["inbound_text", "outbound_text"]]

# Previewing 
print(text.head())

# Saving the real, raw inbound data
text.to_pickle("/kaggle/working/objects/raw_text.pkl")

inbound = text["inbound_text"]
outbound = text["outbound_text"]

In [ ]:
# Showing the results of data wrangling
show(amazon_data)

- Great. So now we are going to build upon this **amazon_data** from now on 
- **Since I am working with unsupervised data for the first part of my chatbot, I do not need a train-test split. My test set would also just end up needing labels as well. For this reason, I will just subset my data to the inbound/outbound data without employing the concept of unseen data.** Pay great attention to this part 

## Text Preprocessing Pipeline 
- In this process, it's expected that I will find much edge cases, and I don't have to address them all. However, in the end, I felt like my prerprocessing pipeline is relatively complete given this dataset 
- Also, this was an iterative process because I fit my models in the next notebook just to notice that my tweets still had '\u200d', '#', '', and '@' even though I still did a lot of fitting already. These are just extra emojis and extra punctuation I decided at a later stage to remove, but in the end I just modified my pipeline function 

- Before that, I did a bit of EDA with bag of words to see what words are being used by the dataset to better preprocess the data and prevent working in too much granularity. 


In [ ]:
# Import Count Vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Writing a function to plot the counts that are in the data 
def top10_bag_of_words(data, output_name, title):
    """ Taking as input the data and plots the top 10 words based on counts in this text data """
    bag_of_words = CountVectorizer()
    inbound = bag_of_words.fit_transform(data)
    
    # Output will be a sparse matrix 
    word_counts = np.array(np.sum(inbound, axis=0)).reshape((-1, ))
    words = np.array(bag_of_words.get_feature_names_out())
    words_df = pd.DataFrame({"word": words, 
                             "count": word_counts})
    words_rank = words_df.sort_values(by="count", ascending=False)
    
    # Storing it in a CSV so that I can inspect it myself 
    words_rank.to_csv("words_rank.csv") # Run this for only the first time and then comment it out 
    words_rank.head()
    
    # Visualizing top 10 words 
    plt.figure(figsize=(12, 6))
    sns.barplot(x=words_rank["word"][ :10], y=words_rank["count"][ :10].astype(int), data=amazon_data, palette="inferno")
    plt.title(title)
    
    # Save the figure 
    # plt.savefig(f"/kaggle/working/visualizations/{output_name}.png")

In [ ]:
top10_bag_of_words(inbound, "most_common_before" , "Top 10 Most Common Words in My Inbound Data Before Preprocessing")

We see that after preprocessing, we will remove the handles which is what applesupport and 115858 is in the visualization above. For now, I briefly visualized the top 10 most common tokens to show that my data as it currently is quite dirty, they will definitely skew and ruin my results if I don't do preprocessing!

### Emoji Analysis
You can leave emojis in and have model learn embeddings for the emojis. Emoji are used in very specific instances, if they use emojis it will be in a useful sense. If the emojis are used for inside jokes, then it's different story, but that is likely not the case for this dataset because these are customer service tweets.

There's actually an embedding called emoji2vec, but that's something I can explore down the line. Right now, I think doc2vec won't be able to handle these emojis in the best way and it might skew my results if I leave them in.

Extracted from [this kernel](https://www.kaggle.com/psbots/customer-support-meets-spacy-universe)

In [ ]:
# Analysis will go here 

In [ ]:
# Emoji Visualization goes here 

## My Entire Preliminary Text Preprocessing Pipeline

I wanted all the preprocessing to be an iterative process, so I compiled it all in one function so that it runs at the same time and hence there is an enhanced sense of organization. At every preprocessing step, I visualize the lengths of each tokens of the data. In general, things like removing stop-words will shift the distribution to the left because we have fewer and fewer tokens at every preprocessing step. I also provide a peek to the head of the data at each step so that it clearly shows what processing is being done at each step.

In [ ]:
# Building tokenizer goes here 
# from spacy_cld import LanguageDetector

### Variable storage
punct_base = ['?']

# Punctuations I want to remove, including the empty token
puncts = ['\u200d', '?', '....','..','...','','@','#', ',', '.', '"', ':', ')', '(', '-', '!', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '*', '+', '\\', 
    '•', '~', '£', '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
    '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
    '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 
    'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
    '¹', '≤', '‡', '√', '!','🅰','🅱']

# Using NLTK's stop words corpus
stopwords.words('english');
stop_words = set(stopwords.words('english')) 

# Found a dictionary of common contractions and colloquial language
contraction_colloq_dict = {
                           "btw": "by the way", "ain't": "is not", "aren't": "are not","can't": "cannot", 
                           "'cause": "because", "could've": "could have", "couldn't": "could not", 
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                           "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                           "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                           "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have", "i'm": "i am", 
                           "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                           "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                           "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                           "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                           "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                           "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
                           "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                           "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                           "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
                           "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                           "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                           "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                           "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", 
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                           "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", 
                           "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", 
                           "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", 
                           "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                           "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", 
                           "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have"
}

# Initializing the lemmatizer 
# Initializing spacy objects 
# nlp_cld = spacy.load("en_core_web_md", exclude=["tagger", "ner"])
# lang_detector = spacy_cld.LanguageDetector()
# nlp_cld.add_pipe(lang_detector)

In [ ]:
import contractions as cm
cm.fix("Can't you believe how it's getting harder to find a decent place to eat? I've been searching for a good restaurant that doesn't cost an arm and a leg. It's not just about the food; it's also about the ambiance, you know? I'd love to try that new place, but it's always packed! I'm thinking we should probably make a reservation in advance. Don't you think so too?")

In [ ]:
list("I am a boy with a big fat butt!!!")

In [ ]:
# My preprocessing functions (defining them here so that I could access them from anywhere in the notebook)
# Capture the hashtags and/or usertags 
# Clean comment text 

tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
### Take care of positional parameters 
def clean_text(
        text, words=True, stops=True, urls=True, tags=True, 
        newLine=True, ellipsis=True, special_chars=True, condensed=True, non_breaking_space=True, 
        character_encodings=True, stopwords=True, only_words=True) -> str:
    
    """ Clean tweets after extracting all hashtags and username tags
    Not comprehensive enough to capture all idiosyncrasies, but works for most of the time
    """
    
    # Capture only words and no numbers
    if words:
        pattern = r"\d"
        text = re.sub(pattern, "", text)
        
    # Remove URLs 
    if urls:
        pattern = "(https\:)*\/*\/*(www\.)?(\w+)(\.\w+)\/*\w*"
        text = re.sub(pattern, "", text)
        
    # Remove tags 
    if tags:
        text = re.sub("@\w+", "", text)
        
    # Replacing one or more occurrences of '\n' with ''
    # Replacing multiple occurrences, i.e., >=2 occurrences with '.'
    if newLine:
        text = re.sub("\n+", "", text)
        text = re.sub(r'\.\s+', '.', text)
        
    # Fix contractions
    if condensed:
        try:
            text = cm.fix(text)
        except: 
            print(text)
        
    # Remove non-breaking space 
    if non_breaking_space: 
        pattern = r"(\xa0|&nbsp)"
        text = re.sub(pattern, "", text)
        
    # Remove stopwords
    if stopwords:
        text = text.lower()
        # print(f"Original Shape of the Data is {.shape}")
        
        # Splitting with NLTK's Tweet tokenizer. This limits repeated characters to 
        # three with the reduce lens parameter and strips all the "@'s". It also splits 
        # it into 1-gram tokens         
        words = tokenizer.tokenize(text)
        filtered_words = [word for word in words if word not in eng_stopwords]
        text = " ".join(words)
        text = text.strip()  # Add further checks for cleaning 
    
    return text

    # Only words
#     if only_words:
#         text = re.sub(r"[^\w\n\.]+", " ", text)
#         text = text.strip()
        

def extract_hashtags(text: str) -> List[str]:
    """ Returns all Twitter hashtags from the text"""
    hashtags_ls = re.findall("#\w+", text)
    return list(set(hashtags_ls))

# Function to detect language using langdetect
## Check if I have to perform sentence level tokenization first 
def detect_language(text: str) -> str:
    """
    Detect the language of the given text using langdetect library.

    Parameters:
    - text (str): The input text for language detection.

    Returns:
    - str: The detected language code (e.g., 'en' for English).
           If language detection fails, returns 'unknown'.
    """
    try:
        # Attempt to detect the language using langdetect
        return detect(text)["lang"]
    
    except Exception:
        # Return 'unknown' if language detection fails
        return "unknown"

### Apply the custom TWEET Tokenizer if required. Check for it 
# End-to-end tokenizer function 
def tokenize_lemmatize(text: str) -> List[str]: 
    ''' I am making my own end-to-end tokenizer'''
    
    # Convert all to lower case 
    data = data.lower()
    print(f"Original Shape of the Data is {data.shape}")
    
    # Splitting with NLTK's Tweet tokenizer. This limits repeated characters to 
    # three with the reduce lens parameter and strips all the "@'s". It also splits 
    # it into 1-gram tokens 
    doc = nlp(text)
    
    # Extract lemmatized text and store it back into a string 
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text 
    
    
def remove_items(tokens: List[str], items_to_remove: List[str]) -> List[str]:
    """
    This function removes a list of items from another list.

    Parameters:
    original_list (list): The list from which items are to be removed.
    items_to_remove (list): The list of items to be removed from the original list.

    Returns:
    list: A new list that contains only the items from the original list that are not in the items to remove.
    """
    
    # Use list comprehension to create a new list that contains only the items from the original list
    # that are not in the items to remove
    return [item for item in original_list if item not in items_to_remove]


def correct_spellings(tokens: List[str]) -> List[str]:
    """
    This function takes as input a list of words and outputs a list of the corrected spelling for each word.

    Parameters:
    tokens (List[str]): The list of words for which the spelling is to be corrected.

    Returns:
    List[str]: A list of corrected words.
    """
    
    # Initialize an empty list to store the corrected words
    corrected_text = []

    # Iterate over each word in the input list
    for word in x:
        
        # Append the corrected word to the corrected_text list
        corrected_text.append(spell.correction(word))
    
    # Return the list of corrected words
    return corrected_text


def get_wordnet_pos(word: str):
    """
    This function determines the WordNet part of speech (POS) tag for a given word.
    This is often used in the process of lemmatization, where words are reduced to their base or root form.

    Parameters:
    word (str): The word for which the POS tag is to be determined.

    Returns:
    str: The WordNet POS tag for the input word.
    """
    
    # Use NLTK's pos_tag function to determine the part of speech of the word
    # pos_tag returns a list of tuples, where each tuple contains a word and its POS tag
    # We're only interested in the POS tag of the first word, so we use [0][1][0] to extract the first character of the POS tag
    # The upper() function is used to convert this character to uppercase
    tag = nltk.pos_tag([word])[0][1][0].upper()

    # This dictionary maps the first character of the POS tag to a corresponding WordNet POS tag
    # The keys "J", "N", "V", and "R" represent adjective, noun, verb, and adverb, respectively
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    # Return the WordNet POS tag corresponding to the input word's POS tag
    # If the first character of the POS tag is not in tag_dict, default to wordnet.NOUN
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_list(tokens: List[str]) -> List[str]:
    '''This lemmatizer function should work on a single list of tokenized data'''
    # Returning a list again
    return [(lemmatizer.lemmatize(token), get_wordnet_pos(token)) for token in tokens]

def extract_emojis(): 
    """Details to be added later"""
    pass 

def limit_length(tokens: List[str], min_tokens: int, max_tokens: int) -> float: 
    ''' Inputs a list and drops it out of the document if 
        the document has more than the max and less than the min'''
    output = x
    if len(x) <= min_tokens:
        output = np.nan
        
    if len(x) > max_tokens:
        output = np.nan
    return output

def remove_from_list(original_list: List[str], items_to_remove: List[str]): 
    """
    Remove a list of items from another list. 
    
    This function takes two lists as input: `original_list` and `items_to_remove`.
    It returns a new list that contains all items from `original_list` that are not in `items_to_remove`.
    
    The function uses a set for `items_to_remove` to optimize the removal operation.
    
    Parameters: 
    original_list (list): The original list 
    items_to_remove (list): The list of items to remove from the original list 
    
    Returns: 
    list: A new list with the items removed. 

    """
    # Convert items_to_remove to a set for faster lookup
    items_to_remove_set = set(items_to_remove)
    
    # Use a list comprehension to create a new list that includes only the items from original_list
    # that are not in items_to_remove_set
    return [item for item in original_list if item not in items_to_remove_set]


def visualize_lengths(data: list, title: str) -> None:
    '''Visualizing lengths of tokens in each tweet'''
    lengths = [len(i) for i in data]
    plt.figure(figsize=(13, 6))
    plt.hist(lengths, bins=40)
    plt.title(title)
    plt.show()

def validate(): 
    # Keep function validation out of scope for the time being 
    pass 


#     # Exapanding contractions and colloquial language 
#     text = text.progress_apply(replace_from_dict, dic=contraction_colloq_dict)
#     print(f"Expanded contractions into extra tokens. Shape is still {text.shape} .\n \n Peek: \n {text.head()}")
#     visualize_lengths(text)
    
#     # Removing non-English tweets with Spacy 
#     text = text[only_english(text)]
#     print(f"Remove all non-English tweets. Shape is {text.shape}. Clearly less than before! \n \n Peek: \n {data.head()}")
    
#     # Lemmatization 
#     text = text.progress_apply(lemmatize_list) 
#     print(f"Lemmatized the tokens. Shape is still {text.shape}. \n \n Peek: \n {text.head()}")
#     visualize_lengths(text, "Length of tokens after step 8")
    
#     # Remove again to make sure I get everything 
#     text = text.progress_apply(remove_from_list, stuff_to_remove=puncts)
    
#     # Using progress bar to show the progress bar 
#     data = data.progress_apply(tokenizer.tokenize)
    
#     # Limiting length of tweets 
#     max_tokens = 50 
#     min_tokens = 5 
#     data = data.progress_apply(limit_length, min_tokens=min_tokens, max_tokens=max_tokens)
#     # Dropping all NaN values, which are the token limits that didn't meet the thresholding requirements 
#     data = da

In [ ]:
clean_text("Hey @user123, great meeting you at the #TechConference2023! Can't believe we discussed over 100 new ideas! #InnovationOverload #TechIsLife'")

In [ ]:
print(type(wordnet.ADJ))

In [ ]:
amazon_data["inbound_lang"] = amazon_data["inbound_text"].apply(detect_language)

In [ ]:
len(list(amazon_data["inbound_text"])[1])

In [ ]:
amazon_data = amazon_data[amazon_data["inbound_lang"]=="en"]
# Visualize data lengths after first pass 
visualize_lengths(list(amazon_data["inbound_text"]), "Lengths post inbound processing")

In [ ]:
amazon_data["inbound_hashtags"] = amazon_data["inbound_text"].progress_apply(extract_hashtags)
amazon_data["outbound_hashtags"] = amazon_data["outbound_text"].progress_apply(extract_hashtags)

In [ ]:
amazon_data.to_picke("")

**Observations**
- Okay, if I filter for any English rows, the follow-up responses besides the given "inbound_text" and "outbound_text" will also be in English 
- After the process, I only have 122340 rows left 

In [ ]:
# end-to-end 
amazon_data["hashtags"] = amazon_data[""]

In [ ]:
from spellchecker import SpellChecker

In [ ]:
# Saving all my results
with open('proceesed_v1.pkl', 'wb') as handle:
    pickle.dump(amazon_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
amazon_data = pd.read_pickle("/kaggle/input/working-data/proceesed_v1.pkl")
amazon_data

In [ ]:
# Applying data cleaning 
amazon_data["cleaned_inbound_text"] = amazon_data["inbound_text"].progress_apply(clean_text)

In [ ]:
amazon_data.head()

## Unit Testing for preprocessing pipeline 

# Length of Words 
# 